# MCP

In [1]:
import os

import chromadb
import dotenv
from agents import Agent, Runner, WebSearchTool, function_tool, trace
from agents.mcp import MCPServerStreamableHttp

dotenv.load_dotenv()

True

Let's set up our RAG database connection:

In [2]:
chroma_client = chromadb.PersistentClient(path="../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [9]:
# This is the same code as in the rag.ipynb notebook


@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

Integrate Open AI Search as an MCP:

In [10]:
# Open AI WebSearchTool:

calorie_agent_with_search = Agent(
    name="Calorie Assistant",
    tools=[
        calorie_lookup_tool,
        WebSearchTool()      
    ]
)


Reference query - shouldn't use WebSearch():

In [11]:
with trace("Nutrition Assistant with MCP - Only uses calorie_lookup_tool"):
    result = await Runner.run(
        calorie_agent_with_search,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result)

RunResult:
- Last agent: Agent(name="Calorie Assistant", ...)
- Final output (str):
    - Rough totals for one medium banana + one medium apple: about 200 kcal.
    - Calories per 100 g:
      - Banana: ~89 kcal
      - Apple: ~52 kcal
    
    If you know the exact weights, I can recalc precisely.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [12]:
result = await Runner.run(calorie_agent_with_search, "Which coffee shop should I go to, taking into account the weather today in Melbourne, Australia?")
print(result.final_output)

Here’s Melbourne today:
- Today (Tue Oct 28): Mostly sunny, high around 18°C, low ~6°C. Currently sunny. Great for a stroll, with a pleasant but cool breeze in the morning/afternoon. 

Coffee shop picks by vibe (Melbourne, Australia)
- If you want a trendy, cafe-with-ambience spot: Proud Mary (Collins St or nearby) — good beans, stylish space, plenty of seating.
- If you prefer expert roasting and a coffee-forward menu: Industry Beans (Melbourne CBD or Fitzroy) — known for meticulous coffee prep and a chic, industrial vibe.
- If you want a classic Melbourne cafe experience with solid espresso and a neighbourhood feel: Axil Coffee Roasters (various locations) — reliable, comfy, great for a relaxed break.

Would you tell me:
- Which area you’ll be near (CBD, Fitzroy, Southbank, etc.)?
- Indoor vs outdoor seating preference?
- Any dietary needs (milk alternatives, pastries, etc.)?

I can tailor a specific recommendation and a quick route.
